In [37]:
import re

import pandas as pd 
import numpy as np 

import matplotlib.pyplot as plt 
import seaborn as sns 

In [57]:
YEARS = [2018, 2019, 2020]
cc_file = 'Corporations/Corporations Responses/Climate Change/{}_Full_Climate_Change_Dataset.csv'
cc = [pd.read_csv(cc_file.format(year)) for year in YEARS]

In [61]:
#dff.pivot(index='row_name', columns='column_name', values='response_value')
df = cc[2]\
    [cc[2]['question_number'] == 'C4.1a']\
    [['row_number', 'column_name', 'response_value', 'organization']]\
    .dropna(subset=['response_value'])
df = df.pivot(index=['organization', 'row_number'], columns='column_name')['response_value']

display(df)
df = df.astype({ col: 'float' for col in [
    'C4.1a_C8Target year', 'C4.1a_C5Base year', 'C4.1a_C9Targeted reduction from base year (%)', 
    'C4.1a_C6Covered emissions in base year (metric tons CO2e)', 'C4.1a_C12% of target achieved [auto-calculated]',
    'C4.1a_C2Year target was set'
]})

# CALCULATE NEW METRICS
df['target_years'] = df['C4.1a_C8Target year'] - df['C4.1a_C2Year target was set'] + 1
df['emissions_reduction_OBJ'] = df['C4.1a_C9Targeted reduction from base year (%)'] / 100 * df['C4.1a_C6Covered emissions in base year (metric tons CO2e)']
df['emissions_reduction_ACT']= df['C4.1a_C12% of target achieved [auto-calculated]'] / 100 * df['C4.1a_C6Covered emissions in base year (metric tons CO2e)']
df['emissions_reduced_per_year_ACT'] = df['emissions_reduction_OBJ'] * df['C4.1a_C12% of target achieved [auto-calculated]'] / 100 / (2020 - df['C4.1a_C2Year target was set'] + 1)
df['years_to_achieve_ACT'] = df['emissions_reduction_OBJ'] / df['emissions_reduced_per_year_ACT']
df['years_diff'] = df['target_years'] - df['years_to_achieve_ACT']

# CLEAN UP VALUES
df['years_diff'].replace(np.inf, 0, inplace=True)
df['years_diff'].replace(np.NINF, 0, inplace=True)
df.loc[df['C4.1a_C12% of target achieved [auto-calculated]'] < 0, 'years_diff'] = np.nan
df.loc[(df['years_diff'] > df['years_diff'].std()) | (df['years_diff'] < -df['years_diff'].std()), 'years_diff'] = np.nan
df['years_diff'].replace(np.inf, 0, inplace=True)
df['years_diff'].replace(np.NINF, 0, inplace=True)
df.loc[df['years_diff'] == 0, 'years_diff'] = 0.00000001
df.loc[df['emissions_reduction_OBJ'] == 0, 'emissions_reduction_OBJ'] = 0.00000001

# WEIGH ALL METRICS INTO ONE PER COMPANY
# "strategy" is weighted by the emissions reduction objective so that bigger strategies have larger sway over the average value
dff = df\
    .groupby(['organization', 'row_number'], as_index=False)\
    .agg({ 
        'years_diff': lambda x: np.average(x, weights=df.loc[x.index, 'emissions_reduction_OBJ']),
        'emissions_reduction_OBJ': 'sum',
        'C4.1a_C6Covered emissions in base year (metric tons CO2e)': 'sum',
        'emissions_reduction_ACT': 'sum' })\
    .rename(columns={ 'C4.1a_C6Covered emissions in base year (metric tons CO2e)': 'emissions_covered' })\
    .dropna()\
    .reset_index()

# CALCULATE RANKS AND PERCENTAGES
dff['pct_OBJ'] = dff['emissions_reduction_OBJ'] / dff['emissions_covered']
dff['pct_ACT'] = dff['emissions_reduction_ACT'] / dff['emissions_covered']
dff['kpi_obj_strategy_rank'] = dff['years_diff'].rank(pct=True)
dff['kpi_obj_ambition_rank'] = dff['pct_OBJ'].rank(pct=True)
dff['kpi_obj_progress_rank'] = dff['pct_ACT'].rank(pct=True)

dff = dff.sort_values(by=['kpi_obj_strategy_rank']).reset_index()

# GRAPH
sns.kdeplot(dff['years_diff'], shade=True)
plt.xlim(-50, 50)

sns.relplot(
    x='kpi_obj_ambition_rank',
    y='years_diff',
    size='emissions_covered',
    sizes=(50, 500),
    hue='emissions_covered',
    data=dff)
plt.xlim(0, 1)
plt.axhline(0, 1, 0, color='r', linestyle='--')

sns.kdeplot(
    x=dff['kpi_obj_ambition_rank'],
    y=dff['kpi_obj_progress_rank'],
    cmap="Blues",
    shade=True,
    bw=0.25)
plt.ylim(0, 1.0)
plt.xlim(0, 1.0)
plt.gca().set_aspect('equal', adjustable='box')
plt.show()

,column_name,C4.1a_C10% achieved (emissions),C4.1a_C11Target status,C4.1a_C12Please explain,C4.1a_C1Target reference number,C4.1a_C2Scope,C4.1a_C3% emissions in Scope,C4.1a_C4% reduction from base year,C4.1a_C5Base year,C4.1a_C6Start year,C4.1a_C7Base year emissions covered by target (metric tons CO2e),C4.1a_C8Target year,C4.1a_C9Is this a science-based target?
organization,row_number,,,,,,,,,,,,
3M Company,1,20.0,Underway,"3M, as a science-based company, supports the p...",Abs 1,Scope 1+2 (location-based),100.0,50.0,2002.0,2015.0,18300000.0,2025.0,"No, but we anticipate setting one in the next ..."
AFLAC Incorporated,1,100.0,Replaced,The goal was achieved as 2017 Scope 2 emission...,Abs 1,Scope 2 (location-based),100.0,50.0,2007.0,2008.0,29765.0,2025.0,"No, and we do not anticipate setting one in th..."
AK Steel Holding Corporation,1,18.8,Underway,Target and ongoing emissions reflect those ba...,Abs 1,Scope 1,100.0,18.1,2015.0,2016.0,8092985.0,2025.0,"No, but we anticipate setting one in the next ..."
ARTESYN EMBEDDED TECHNOLOGIES,1,100.0,Underway,"Artesyn has set a goal to reduce its Scope 2, ...",Abs 1,Scope 2 (location-based),100.0,2.5,2014.0,2014.0,100.0,2020.0,"No, but we anticipate setting one in the next ..."
AT&T Inc.,1,100.0,Replaced,We have set an absolute Scope 1 GHG emissions ...,Abs 1,Scope 1,100.0,20.0,2008.0,2011.0,1354054.0,2020.0,"No, and we do not anticipate setting one in th..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Xilinx Inc,1,30.0,Underway,We publically report annual progress on meetin...,Abs 1,Scope 1+2 (location-based),100.0,10.0,2014.0,2014.0,27822.0,2019.0,"No, and we do not anticipate setting one in th..."
"Yum! Brands, Inc.",1,100.0,Underway,Since 2005 we have demonstrated a steady track...,Abs 1,Scope 1+2 (location-based),100.0,22.0,2005.0,2006.0,1943203.0,2017.0,"No, but we anticipate setting one in the next ..."
Zatkoff Seals and Packing,1,100.0,Underway,Our goal is written to be equal to or less the...,Abs 1,Scope 1+2 (location-based),100.0,45.0,2008.0,2008.0,978.35,2020.0,"Yes, this target has been approved as science-..."


KeyError: 'Only a column name can be used for the key in a dtype mappings argument.'